# Set up

In [1]:
# update path with data dir
import sys
sys.path.append('../../optimiser/')

In [2]:
import pandas as pd
import patsy
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import PredefinedSplit, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import optimiser
import scipy.stats
import itertools

[MLENS] backend: threading


# Getting data

In [3]:
file_name = '../csvs/element_gameweek_features_v06.csv'

In [4]:
element_gameweek_df = pd.read_csv(file_name)

In [5]:
element_gameweek_df.fillna(
    element_gameweek_df[element_gameweek_df['event'] <= 20].mean(),
    inplace=True
)

In [6]:
all_element_gameweek_df = element_gameweek_df.copy()

In [7]:
all_element_gameweek_df['predicted_total_points'] = 0

In [8]:
[i for i in list(element_gameweek_df.columns) if i not in [
     'safe_web_name',
     'element',
     'event',
     'fixture',
     'team',
     'opposition_team',
     'element_type',
     'minutes',
     'rolling_value_points',
     'total_points',
     
     'expected_total_points_against_opposition_team',
     'rolling_avg_total_points_element_home',
     'rolling_max_total_points_element',
     'rolling_avg_bps_element',
     'expected_total_points_element_home_away',
     'rolling_avg_total_points_element_away',
     'rolling_avg_total_points_own_team',
     'rolling_avg_total_points_against_opposition_team',
     'rolling_avg_minutes_element',
     'rolling_avg_total_points_opposition_team',
     'rolling_avg_dribbles_element',
     'rolling_avg_total_points_against_opposition_team_element_type',
     'rolling_avg_total_points_against_opposition_team_element_type_away',
     'rolling_avg_total_points_against_opposition_team_element_type_home',
     'rolling_avg_goals_conceded_element',
     'rolling_avg_completed_passes_element',
     'rolling_avg_total_points_element_type',
     'rolling_avg_recoveries_element',
]]

['was_home',
 'is_sunday',
 'is_weekday',
 'is_early',
 'is_late',
 'value',
 'rolling_avg_total_points_element',
 'has_high_points_ever_element',
 'rolling_avg_bonus_element',
 'rolling_avg_goals_scored_element',
 'rolling_avg_clean_sheets_element',
 'rolling_avg_assists_element',
 'rolling_avg_key_passes_element',
 'rolling_avg_winning_goals_element',
 'rolling_avg_tackled_element',
 'rolling_avg_clearances_blocks_interceptions_element',
 'rolling_avg_big_chances_created_element',
 'rolling_avg_target_missed_element',
 'rolling_avg_fouls_element',
 'rolling_avg_tackles_element',
 'rolling_avg_open_play_crosses_element',
 'rolling_avg_offside_element',
 'rolling_avg_big_chances_missed_element',
 'rolling_avg_saves_element',
 'rolling_avg_attempted_passes_element',
 'rolling_avg_errors_leading_to_goal_element',
 'rolling_avg_errors_leading_to_goal_attempt_element',
 'rolling_avg_own_goals_element',
 'rolling_avg_penalties_concededd_element',
 'rolling_avg_penalties_missed_element',
 'r

# Evaluating models

## Helpers

### Modelling

In [9]:
formula = \
'total_points ~ C(element_type) + C(team) + C(opposition_team) + ' + \
' + '.join([i for i in element_gameweek_df.columns if i not in
 [
     'safe_web_name',
     'element',
     'event',
     'fixture',
     'team',
     'opposition_team',
     'element_type',
     'minutes',
     'rolling_value_points',
     'total_points',
     
     'expected_total_points_against_opposition_team',
     'rolling_avg_total_points_element_home',
     'rolling_max_total_points_element',
     'rolling_avg_bps_element',
     'expected_total_points_element_home_away',
     'rolling_avg_total_points_element_away',
     'rolling_avg_total_points_own_team',
     'rolling_avg_total_points_against_opposition_team',
     'rolling_avg_minutes_element',
     'rolling_avg_total_points_opposition_team',
     'rolling_avg_dribbles_element',
     'rolling_avg_total_points_against_opposition_team_element_type',
     'rolling_avg_total_points_against_opposition_team_element_type_away',
     'rolling_avg_total_points_against_opposition_team_element_type_home',
     'rolling_avg_goals_conceded_element',
     'rolling_avg_completed_passes_element',
     'rolling_avg_total_points_element_type',
     'rolling_avg_recoveries_element',
 ]])

In [10]:
scaled_feature_cols = [
    i for i in element_gameweek_df.columns if i not in [
        'safe_web_name', 'element', 'event', 'fixture', 'team',
        'opposition_team', 'element_type', 'was_home', 'is_sunday',
        'is_weekday', 'is_late', 'is_early', 'has_high_points_ever_element',
        'total_points', 'minutes', 'rolling_value_points'
    ]
]

In [11]:
def calculate_expected_total_points_against_opposition_team_home_away(row):
    a = 'rolling_avg_total_points_against_opposition_team_element_type_home'
    b = 'rolling_avg_total_points_against_opposition_team_element_type_away'
    c = 'rolling_avg_total_points_element'
    d = 'rolling_avg_total_points_element_type'
    if row['was_home'] == 1:
        return row[a] * row[c] / row[d]
    else:
        return row[b] * row[c] / row[d]


def munge_data(df, e, prediction_events, minute_threshold=60):
    # filter weeks after test week
    event_df = df.copy()
    event_df = event_df[event_df['event'] <= e + prediction_events - 1]

    cols = event_df.columns

    # columns that we can fill down
    unknown_element_cols = [
        'was_home',
        'is_sunday',
        'is_weekday',
        'is_early',
        'is_late',
        'value',
        'rolling_avg_total_points_element',
        'has_high_points_ever_element',
        'rolling_avg_bonus_element',
        'rolling_avg_goals_scored_element',
        'rolling_avg_clean_sheets_element',
        'rolling_avg_assists_element',
        'rolling_avg_key_passes_element',
        'rolling_avg_winning_goals_element',
        'rolling_avg_tackled_element',
        'rolling_avg_clearances_blocks_interceptions_element',
        'rolling_avg_big_chances_created_element',
        'rolling_avg_target_missed_element',
        'rolling_avg_fouls_element',
        'rolling_avg_tackles_element',
        'rolling_avg_open_play_crosses_element',
        'rolling_avg_offside_element',
        'rolling_avg_big_chances_missed_element',
        'rolling_avg_saves_element',
        'rolling_avg_attempted_passes_element',
        'rolling_avg_errors_leading_to_goal_element',
        'rolling_avg_errors_leading_to_goal_attempt_element',
        'rolling_avg_own_goals_element',
        'rolling_avg_penalties_concededd_element',
        'rolling_avg_penalties_missed_element',
        'rolling_avg_penalties_saved_element',
        'rolling_avg_red_cards_element',
        'rolling_avg_yellow_cards_element',
        'rolling_avg_minutes_element_p3',
        'rolling_avg_total_points_element_p3',
        'rolling_avg_goals_scored_opposition_team',
        'rolling_avg_goals_conceded_opposition_team',
        'rolling_avg_total_points_element_type',
    ]

    # columns we need to look up
    unknown_opposition_cols = [
        'rolling_avg_goals_scored_opposition_team',
        'rolling_avg_goals_conceded_opposition_team',
        'rolling_avg_total_points_against_opposition_team_element_type_home',
        'rolling_avg_total_points_against_opposition_team_element_type_away',
    ]

    # columns we need to calculate
    unknown_engineered_cols = [
        'expected_total_points_against_opposition_team_home_away',
    ]

    # fill in nans for future data we wouldn't know
    event_df.loc[event_df['event'] > e,
                 unknown_element_cols + unknown_opposition_cols + unknown_engineered_cols
                ] = np.nan
    event_df.sort_values(['element', 'event', 'fixture'], inplace=True)
    # fill down the element data
    event_df[unknown_element_cols] = event_df[unknown_element_cols].fillna(method='ffill')

    # create look up tables for opposition team data
    # we have to look two events back, as some teams won't have played last event
    against_opposition_event_df_1 = event_df[event_df['event'] == e][
        ['opposition_team','element_type','event'] + unknown_opposition_cols].drop_duplicates()


    against_opposition_event_df_2 = event_df[event_df['event'] == e - 1][
        ['opposition_team','element_type','event'] + unknown_opposition_cols].drop_duplicates()

    against_opposition_event_df = pd.concat([against_opposition_event_df_1, against_opposition_event_df_2])

    # get the most recent opposition team data
    against_opposition_event_df = against_opposition_event_df.join(
        against_opposition_event_df.groupby(['opposition_team', 'element_type'])['event'].max(),
        on=['opposition_team', 'element_type'],
        rsuffix='_most_recent')

    against_opposition_event_df = \
    against_opposition_event_df[against_opposition_event_df['event'] == against_opposition_event_df['event_most_recent']]

    event_df = event_df.join(
        against_opposition_event_df.set_index(['opposition_team', 'element_type']),
        on=['opposition_team', 'element_type'],
        rsuffix='_fill')

    # fill in opposition team data from lookup table
    for i in unknown_opposition_cols:
        event_df.loc[event_df['event'] > e, i] = event_df[event_df['event'] > e][i+'_fill']

    event_df['expected_total_points_against_opposition_team_home_away'] = \
    event_df.apply(calculate_expected_total_points_against_opposition_team_home_away, axis=1)

    # filter for frequently appearing players
    event_df = event_df[event_df['rolling_avg_minutes_element_p3'] >= minute_threshold][cols]
    
    return event_df


def split_data(event_df, last_train_event, last_test_event):
    # define train-test split
    test_fold = [-1 if i <= last_train_event else 0 for i in event_df['event'] if i <= last_test_event]
    ps = PredefinedSplit(test_fold)

    # split df into train and test
    for train_index, test_index in ps.split():
        event_df_train, event_df_test = \
        event_df.copy().iloc[train_index], event_df.copy().iloc[test_index]
    
    return event_df_train, event_df_test, ps


def standardise_data(event_df, event_df_train, event_df_test, scaled_feature_cols):
    scale_train = event_df_train.copy()
    scale_test = event_df_test.copy()
    scale_df = event_df.copy()
    scaled_event_df_train = event_df_train.copy()
    scaled_event_df_test = event_df_test.copy()
    scaled_event_df = event_df.copy()

    scaler = StandardScaler().fit(scale_train[scaled_feature_cols].values)

    scale_train = scaler.transform(scale_train[scaled_feature_cols].values)
    scale_test = scaler.transform(scale_test[scaled_feature_cols].values)
    scale_df = scaler.transform(scale_df[scaled_feature_cols].values)

    scaled_event_df_train[scaled_feature_cols] = scale_train
    scaled_event_df_test[scaled_feature_cols] = scale_test
    scaled_event_df[scaled_feature_cols] = scale_df
    
    return scaled_event_df_train, scaled_event_df_test, scaled_event_df


def get_pcs(event_X_train, event_X_test, n_categorical_features):
    pca = PCA()
    pca.fit(event_X_train[:,(n_categorical_features + 1):])
    
    event_X_train_pca = np.concatenate(
        (
            event_X_train[:,:(n_categorical_features + 1)],
            pca.transform(event_X_train[:,(n_categorical_features + 1):])
        ), axis=1)
    
    event_X_test_pca = np.concatenate(
        (
            event_X_test[:,:(n_categorical_features + 1)],
            pca.transform(event_X_test[:,(n_categorical_features + 1):])
        ), axis=1)
    
    return event_X_train_pca, event_X_test_pca


def split_matrices(event_X, event_y, ps):
    for train_index, test_index in ps.split():
        event_X_train, event_X_test = event_X[train_index], event_X[test_index]
        event_y_train, event_y_test = event_y[train_index], event_y[test_index]
    
    return event_X_train, event_X_test, event_y_train, event_y_test


def select_features(event_X, event_X_train, event_X_test, features_index):    
    event_X_train_sel = event_X_train[:,features_index]
    event_X_test_sel = event_X_test[:,features_index]
    event_X_sel = event_X[:,features_index]
    
    return event_X_train_sel, event_X_test_sel, event_X_sel


def retune_model(event_df,
                 last_train_event,
                 last_validation_event,
                 standardise,
                 features_index,
                 model,
                 parameter_space,
                 n_iter_tune=100):
    
    # split data into train and validation set
    event_df_train, event_df_validation, ps_tune = split_data(event_df, last_train_event, last_validation_event)

    # standardise appropriate variables if necessary
    scaled_event_df = event_df.copy()
    if standardise:
        scaled_event_df_train, scaled_event_df_validation, scaled_event_df = \
        standardise_data(event_df, event_df_train, event_df_validation, scaled_feature_cols)        

    # get reponse vector and feature matrix
    event_y, event_X = patsy.dmatrices(formula, scaled_event_df, return_type='matrix')

    # split response vector and feature matrix into train and test
    event_X_train, event_X_validation, event_y_train, event_y_validation = \
    split_matrices(event_X, event_y, ps_tune)

    # if only certain features selected, get their indices
    event_X_train_sel = event_X_train
    event_X_validation_sel = event_X_validation
    event_X_sel = event_X
    if features_index:
        event_X_train_sel, event_X_validation_sel, event_X_sel = \
        select_features(event_X, event_X_train, event_X_validation, features_index)
    
    # search hyperparameter space
    tuner = RandomizedSearchCV(
        model,
        parameter_space,
        n_iter=n_iter_tune,
        scoring='neg_mean_squared_error',
        refit=True,
        cv=ps_tune,
        error_score=100, n_jobs=1)
    
    tuner.fit(event_X_sel, event_y.ravel())
    
    return tuner.best_estimator_

### Predicting points

In [12]:
def predict_test_set(df,
                     model,
                     parameter_space=False,
                     prediction_events=1,
                     prediction_weight=1,
                     features_index=False,
                     standardise=False,
                     pcs=False,
                     start=21,
                     end=38,
                     n_iter_tune=100,
                     verbose=0):
    y_pred_arr = []
    y_test_arr = []
    event_df_test_arr = []

    # for each event we want to predict
    for e in range(start, end + 1):
        if verbose > 0:
            print('predicting event', e)
        
        # munge data
        event_df = munge_data(df, e, prediction_events, minute_threshold=60)

        # split df into train and test
        event_df_train, event_df_test, ps = split_data(event_df, e - 1, 38)
        
        # standardise appropriate variables if necessary
        scaled_event_df = event_df.copy()
        if standardise:
            scaled_event_df_train, scaled_event_df_test, scaled_event_df = \
            standardise_data(event_df, event_df_train, event_df_test, scaled_feature_cols)        

        # get reponse vector and feature matrix
        event_y, event_X = patsy.dmatrices(formula, scaled_event_df, return_type='matrix')
        
        # split response vector and feature matrix into train and test
        event_X_train, event_X_test, event_y_train, event_y_test = \
        split_matrices(event_X, event_y, ps)
        
        # get pcs if necessary
        if pcs:
            n_categorical_features = event_X.design_info.column_names.index('value') - 1
            event_X_train, event_X_test = get_pcs(event_X_train, event_X_test, n_categorical_features)
        
        # if only certain features selected, get their indices
        event_X_train_sel = event_X_train
        event_X_test_sel = event_X_test
        event_X_sel = event_X
        if features_index:
            event_X_train_sel, event_X_test_sel, event_X_sel = \
            select_features(event_X, event_X_train, event_X_test, features_index)
            
        # retune hyperparameters
        if parameter_space:
            model = retune_model(event_df,
                                 e - 6,
                                 e - 1,
                                 standardise,
                                 features_index,
                                 model,
                                 parameter_space,
                                 n_iter_tune)

        # fit model on training data
        model.fit(event_X_train_sel, event_y_train.ravel())
        # predict test event
        event_y_pred = model.predict(event_X_test_sel).flatten()
        
        # collect predictions and observations 
        y_pred_arr.append(event_y_pred)
        y_test_arr.append(event_y_test)
        
        event_df_test['predicted_total_points'] = event_y_pred
        event_df_test['prediction_event'] = e
        
        event_df_test_arr.append(event_df_test)
        
    
    return np.concatenate(y_pred_arr).ravel(), np.concatenate(y_test_arr).ravel(), pd.concat(event_df_test_arr)

### Constructing teams

In [13]:
def get_event_players(
    df,
    e,
    prediction_events,
    optimise_key,
    prediction_weight):
    
    event_players = df.copy()
    
    event_players = \
    event_players[
        (event_players['prediction_event'] == e)
        & (event_players['event'] <= e + prediction_events - 1)
    ]        
    present_elements = event_players['element'].drop_duplicates().values
    all_df = all_element_gameweek_df.copy()
    all_df = all_df[all_df['event'] == e]
    all_df['prediction_event'] = e
    event_players = pd.concat([event_players, all_df[~all_df['element'].isin(present_elements)]])
    
    
    event_players['event_diff'] = event_players['event'] - event_players['prediction_event']
    event_players['prediction_weight'] = prediction_weight**(event_players['event_diff'])
    event_players['optimise_key_weighted'] = event_players['prediction_weight'] * event_players[optimise_key]
    
    event_players_df = event_players.copy()
    
    event_players_group = event_players.groupby('element')['optimise_key_weighted'].sum()
    event_players = event_players[['element', 'value', 'element_type', 'team']].drop_duplicates()
    event_players = event_players.join(event_players_group, on='element')
    event_players = event_players.to_dict('records')
    
    return event_players, event_players_df

In [14]:
def construct_event_teams_from_existing(df,
                                        prediction_events=1,
                                        prediction_weight=1,
                                        from_scratch_prediction_events=1,
                                        optimise_key='predicted_total_points',
                                        total_budget=1000,
                                        formation=[1, 5, 4, 1],
                                        start=21,
                                        end=38,
                                        transfer_penalty=4,
                                        transfer_limit=11,
                                        verbose=0):
    first_team_arr = []
    bench_arr = []
    team_total_points_arr = []
    predicted_team_total_points_arr = []
    team_df_arr = []
    transfers_arr = []
    carried_over_transfers = 0
    
    for e in range(start, end + 1):
        if verbose > 0:
            print('predicting event', e)
        
        event_players, event_players_df = get_event_players(
            df,
            e,
            prediction_events,
            optimise_key,
            prediction_weight
        )
        
        from_scratch_event_players, _ = get_event_players(
            df,
            e,
            from_scratch_prediction_events,
            optimise_key,
            prediction_weight
        )
        
        if e == start:
            try:
                event_first_team, event_bench = \
                optimiser.construct_optimal_team_from_scratch(
                    from_scratch_event_players,
                    optimise_key='optimise_key_weighted',
                    total_budget=total_budget,
                    formation=formation,
                )

                first_team_arr.append(event_first_team)
                bench_arr.append(event_bench)
                transfers_arr.append([set(), set()])
                
                event_num_transfers = 1
            except Exception as ex:
                print(e, ex)
                first_team_arr.append([])
                bench_arr.append([])
                transfers_arr.append([])
                
                event_num_transfers = 1
        else:
            try:
                event_first_team, event_bench, event_transfers = \
                optimiser.construct_optimal_team_from_existing(
                    event_players,
                    event_first_team,
                    event_bench,
                    total_budget=total_budget,
                    optimise_key='optimise_key_weighted',
                    transfer_penalty=transfer_penalty,
                    transfer_limit=transfer_limit
                )

                first_team_arr.append(event_first_team)
                bench_arr.append(event_bench)
                transfers_arr.append(event_transfers)
                
                event_num_transfers = len(event_transfers[0])
            except Exception as ex:
                print(e, ex)
                first_team_arr.append(event_first_team)
                bench_arr.append(event_bench)
                transfers_arr.append([set(), set()])
                
                event_num_transfers = 0
        
        event_num_transfers = max(event_num_transfers - carried_over_transfers, 0)
        
        event_team_total_points, event_team_predicted_total_points, event_team_df = \
        optimiser.calculate_team_total_points(event_players_df,
                                              event_first_team,
                                              event_bench,
                                              e,
                                              event_num_transfers,
                                              carried_over_transfers)
        
        if event_num_transfers == 0 and carried_over_transfers == 0:
            carried_over_transfers = 1
        if event_num_transfers in (0, 1) and carried_over_transfers == 1:
            carried_over_transfers = 1
        if event_num_transfers == 1 and carried_over_transfers == 0:
            carried_over_transfers = 0
        if event_num_transfers > 1:
            carried_over_transfers = 0


        team_total_points_arr.append(event_team_total_points)
        predicted_team_total_points_arr.append(event_team_predicted_total_points)
        team_df_arr.append(event_team_df)
    
    return (
        first_team_arr, bench_arr,
        team_total_points_arr,
        predicted_team_total_points_arr,
        team_df_arr,
        transfers_arr
    )

## Constructing teams

### Ridge regression

In [15]:
ridge_model = Ridge(alpha=1e-8)

In [16]:
ridge_features_index = [
    False, False, False, False, False, False, False, False, False, False,
    False, False, False, False, True, False, False, False, False, False, False,
    False, False, False, False, False, False, False, False, False, True, False,
    False, True, False, False, False, False, False, False, False, False, False,
    False, False, False, False, True, False, False, False, False, False, False,
    True, False, False, False, False, False, False, False, False, False, True,
    True, False, False, False, False, False, False, False, False, False, True,
    False, True, False, True
]

In [18]:
ridge_y_pred, ridge_y_test, ridge_df_test = predict_test_set(
    element_gameweek_df,
    ridge_model,
    standardise=True,
    features_index=ridge_features_index,
    pcs=False,
    prediction_events=5,
    verbose=1,
    start=16,
    end=20
)

predicting event 16
predicting event 17
predicting event 18
predicting event 19
predicting event 20


In [19]:
ridge_test_loss = (
    mean_absolute_error(ridge_y_test, ridge_y_pred),
    mean_squared_error(ridge_y_test, ridge_y_pred),
    r2_score(ridge_y_test, ridge_y_pred)
)
ridge_test_loss

(2.164181998048021, 8.817528642087087, 0.11223648493579785)

In [29]:

prediction_events_arr = [1, 2, 3, 4, 5]
from_scratch_prediction_events_arr = [1, 2, 3, 4, 5]
formation_arr = [
    [1, 5, 4, 1],
    [1, 5, 3, 2],
    [1, 3, 4, 3],
    [1, 4, 3, 3],
    [1, 3, 5, 2],
    [1, 4, 5, 1],
    [1, 4, 4, 2],
    [1, 5, 2, 3],
]
transfer_limit_arr = [1, 2, 3, 4, 11]

parameters_arr = list(itertools.product(
    prediction_events_arr,
    from_scratch_prediction_events_arr,
    formation_arr,
    transfer_limit_arr,
))

len(parameters_arr)

1000

In [52]:
results_arr = []

for i in parameters_arr:
    print('iteration ', parameters_arr.index(i))
    prediction_events = i[0]
    from_scratch_prediction_events = i[1]
    formation = i[2]
    transfer_limit = i[3]
    
    d = {
        'prediction_events': prediction_events,
        'from_scratch_prediction_events': from_scratch_prediction_events,
        'formation': formation,
        'transfer_limit': transfer_limit,
    }
    
    print(d)
    
    (
        ridge_first_team_arr,
        ridge_bench_arr,
        ridge_total_points_arr,
        ridge_predicted_total_points_arr,
        ridge_team_df_arr,
        ridge_transfers_arr
    ) = \
    construct_event_teams_from_existing(
        ridge_df_test,
        total_budget=1000,
        prediction_events=prediction_events,
        from_scratch_prediction_events=from_scratch_prediction_events,
        formation=formation,
        transfer_penalty=4,
        transfer_limit=transfer_limit,
        start=16,
        end=20,
        verbose=0
    )


    ridge_total_points = sum(ridge_total_points_arr)
    ridge_mean_transfers = np.mean([len(ridge_transfers_arr[i][0]) for i in range(0, len(ridge_transfers_arr))])
    
    d['total_points'] = ridge_total_points
    d['mean_transfers'] = ridge_mean_transfers
    
    results_arr.append(d)
    
    print(ridge_total_points, ridge_mean_transfers)

iteration  0
{'prediction_events': 1, 'from_scratch_prediction_events': 1, 'formation': [1, 5, 4, 1], 'transfer_limit': 1}
346 0.2
iteration  1
{'prediction_events': 1, 'from_scratch_prediction_events': 1, 'formation': [1, 5, 4, 1], 'transfer_limit': 2}
346 0.2
iteration  2
{'prediction_events': 1, 'from_scratch_prediction_events': 1, 'formation': [1, 5, 4, 1], 'transfer_limit': 3}
346 0.2
iteration  3
{'prediction_events': 1, 'from_scratch_prediction_events': 1, 'formation': [1, 5, 4, 1], 'transfer_limit': 4}
346 0.2
iteration  4
{'prediction_events': 1, 'from_scratch_prediction_events': 1, 'formation': [1, 5, 4, 1], 'transfer_limit': 11}
346 0.2
iteration  5
{'prediction_events': 1, 'from_scratch_prediction_events': 1, 'formation': [1, 5, 3, 2], 'transfer_limit': 1}
352 0.2
iteration  6
{'prediction_events': 1, 'from_scratch_prediction_events': 1, 'formation': [1, 5, 3, 2], 'transfer_limit': 2}
352 0.2
iteration  7
{'prediction_events': 1, 'from_scratch_prediction_events': 1, 'format

354 0.4
iteration  63
{'prediction_events': 1, 'from_scratch_prediction_events': 2, 'formation': [1, 3, 5, 2], 'transfer_limit': 4}
354 0.4
iteration  64
{'prediction_events': 1, 'from_scratch_prediction_events': 2, 'formation': [1, 3, 5, 2], 'transfer_limit': 11}
354 0.4
iteration  65
{'prediction_events': 1, 'from_scratch_prediction_events': 2, 'formation': [1, 4, 5, 1], 'transfer_limit': 1}
366 0.6
iteration  66
{'prediction_events': 1, 'from_scratch_prediction_events': 2, 'formation': [1, 4, 5, 1], 'transfer_limit': 2}
366 0.6
iteration  67
{'prediction_events': 1, 'from_scratch_prediction_events': 2, 'formation': [1, 4, 5, 1], 'transfer_limit': 3}
366 0.6
iteration  68
{'prediction_events': 1, 'from_scratch_prediction_events': 2, 'formation': [1, 4, 5, 1], 'transfer_limit': 4}
366 0.6
iteration  69
{'prediction_events': 1, 'from_scratch_prediction_events': 2, 'formation': [1, 4, 5, 1], 'transfer_limit': 11}
366 0.6
iteration  70
{'prediction_events': 1, 'from_scratch_prediction_ev

353 0.6
iteration  125
{'prediction_events': 1, 'from_scratch_prediction_events': 4, 'formation': [1, 5, 3, 2], 'transfer_limit': 1}
342 0.6
iteration  126
{'prediction_events': 1, 'from_scratch_prediction_events': 4, 'formation': [1, 5, 3, 2], 'transfer_limit': 2}
342 0.6
iteration  127
{'prediction_events': 1, 'from_scratch_prediction_events': 4, 'formation': [1, 5, 3, 2], 'transfer_limit': 3}
342 0.6
iteration  128
{'prediction_events': 1, 'from_scratch_prediction_events': 4, 'formation': [1, 5, 3, 2], 'transfer_limit': 4}
342 0.6
iteration  129
{'prediction_events': 1, 'from_scratch_prediction_events': 4, 'formation': [1, 5, 3, 2], 'transfer_limit': 11}
342 0.6
iteration  130
{'prediction_events': 1, 'from_scratch_prediction_events': 4, 'formation': [1, 3, 4, 3], 'transfer_limit': 1}
354 0.4
iteration  131
{'prediction_events': 1, 'from_scratch_prediction_events': 4, 'formation': [1, 3, 4, 3], 'transfer_limit': 2}
354 0.4
iteration  132
{'prediction_events': 1, 'from_scratch_predic

339 0.2
iteration  187
{'prediction_events': 1, 'from_scratch_prediction_events': 5, 'formation': [1, 4, 5, 1], 'transfer_limit': 3}
339 0.2
iteration  188
{'prediction_events': 1, 'from_scratch_prediction_events': 5, 'formation': [1, 4, 5, 1], 'transfer_limit': 4}
339 0.2
iteration  189
{'prediction_events': 1, 'from_scratch_prediction_events': 5, 'formation': [1, 4, 5, 1], 'transfer_limit': 11}
339 0.2
iteration  190
{'prediction_events': 1, 'from_scratch_prediction_events': 5, 'formation': [1, 4, 4, 2], 'transfer_limit': 1}
336 0.4
iteration  191
{'prediction_events': 1, 'from_scratch_prediction_events': 5, 'formation': [1, 4, 4, 2], 'transfer_limit': 2}
336 0.4
iteration  192
{'prediction_events': 1, 'from_scratch_prediction_events': 5, 'formation': [1, 4, 4, 2], 'transfer_limit': 3}
336 0.4
iteration  193
{'prediction_events': 1, 'from_scratch_prediction_events': 5, 'formation': [1, 4, 4, 2], 'transfer_limit': 4}
336 0.4
iteration  194
{'prediction_events': 1, 'from_scratch_predic

325 0.6
iteration  249
{'prediction_events': 2, 'from_scratch_prediction_events': 2, 'formation': [1, 5, 3, 2], 'transfer_limit': 11}
325 0.6
iteration  250
{'prediction_events': 2, 'from_scratch_prediction_events': 2, 'formation': [1, 3, 4, 3], 'transfer_limit': 1}
359 0.6
iteration  251
{'prediction_events': 2, 'from_scratch_prediction_events': 2, 'formation': [1, 3, 4, 3], 'transfer_limit': 2}
359 0.6
iteration  252
{'prediction_events': 2, 'from_scratch_prediction_events': 2, 'formation': [1, 3, 4, 3], 'transfer_limit': 3}
359 0.6
iteration  253
{'prediction_events': 2, 'from_scratch_prediction_events': 2, 'formation': [1, 3, 4, 3], 'transfer_limit': 4}
359 0.6
iteration  254
{'prediction_events': 2, 'from_scratch_prediction_events': 2, 'formation': [1, 3, 4, 3], 'transfer_limit': 11}
359 0.6
iteration  255
{'prediction_events': 2, 'from_scratch_prediction_events': 2, 'formation': [1, 4, 3, 3], 'transfer_limit': 1}
362 0.6
iteration  256
{'prediction_events': 2, 'from_scratch_predi

...

354 1.0
iteration  732
{'prediction_events': 4, 'from_scratch_prediction_events': 4, 'formation': [1, 3, 4, 3], 'transfer_limit': 3}
354 1.0
iteration  733
{'prediction_events': 4, 'from_scratch_prediction_events': 4, 'formation': [1, 3, 4, 3], 'transfer_limit': 4}
354 1.0
iteration  734
{'prediction_events': 4, 'from_scratch_prediction_events': 4, 'formation': [1, 3, 4, 3], 'transfer_limit': 11}
354 1.0
iteration  735
{'prediction_events': 4, 'from_scratch_prediction_events': 4, 'formation': [1, 4, 3, 3], 'transfer_limit': 1}
326 0.6
iteration  736
{'prediction_events': 4, 'from_scratch_prediction_events': 4, 'formation': [1, 4, 3, 3], 'transfer_limit': 2}
343 1.0
iteration  737
{'prediction_events': 4, 'from_scratch_prediction_events': 4, 'formation': [1, 4, 3, 3], 'transfer_limit': 3}
343 1.0
iteration  738
{'prediction_events': 4, 'from_scratch_prediction_events': 4, 'formation': [1, 4, 3, 3], 'transfer_limit': 4}
343 1.0
iteration  739
{'prediction_events': 4, 'from_scratch_predic

360 1.4
iteration  794
{'prediction_events': 4, 'from_scratch_prediction_events': 5, 'formation': [1, 4, 4, 2], 'transfer_limit': 11}
360 1.4
iteration  795
{'prediction_events': 4, 'from_scratch_prediction_events': 5, 'formation': [1, 5, 2, 3], 'transfer_limit': 1}
343 0.6
iteration  796
{'prediction_events': 4, 'from_scratch_prediction_events': 5, 'formation': [1, 5, 2, 3], 'transfer_limit': 2}
334 0.8
iteration  797
{'prediction_events': 4, 'from_scratch_prediction_events': 5, 'formation': [1, 5, 2, 3], 'transfer_limit': 3}
330 1.0
iteration  798
{'prediction_events': 4, 'from_scratch_prediction_events': 5, 'formation': [1, 5, 2, 3], 'transfer_limit': 4}
330 1.0
iteration  799
{'prediction_events': 4, 'from_scratch_prediction_events': 5, 'formation': [1, 5, 2, 3], 'transfer_limit': 11}
330 1.0
iteration  800
{'prediction_events': 5, 'from_scratch_prediction_events': 1, 'formation': [1, 5, 4, 1], 'transfer_limit': 1}
366 0.4
iteration  801
{'prediction_events': 5, 'from_scratch_predi

362 0.6
iteration  856
{'prediction_events': 5, 'from_scratch_prediction_events': 2, 'formation': [1, 4, 3, 3], 'transfer_limit': 2}
374 1.0
iteration  857
{'prediction_events': 5, 'from_scratch_prediction_events': 2, 'formation': [1, 4, 3, 3], 'transfer_limit': 3}
374 1.0
iteration  858
{'prediction_events': 5, 'from_scratch_prediction_events': 2, 'formation': [1, 4, 3, 3], 'transfer_limit': 4}
374 1.0
iteration  859
{'prediction_events': 5, 'from_scratch_prediction_events': 2, 'formation': [1, 4, 3, 3], 'transfer_limit': 11}
374 1.0
iteration  860
{'prediction_events': 5, 'from_scratch_prediction_events': 2, 'formation': [1, 3, 5, 2], 'transfer_limit': 1}
351 0.6
iteration  861
{'prediction_events': 5, 'from_scratch_prediction_events': 2, 'formation': [1, 3, 5, 2], 'transfer_limit': 2}
363 0.6
iteration  862
{'prediction_events': 5, 'from_scratch_prediction_events': 2, 'formation': [1, 3, 5, 2], 'transfer_limit': 3}
363 0.6
iteration  863
{'prediction_events': 5, 'from_scratch_predic

314 1.2
iteration  918
{'prediction_events': 5, 'from_scratch_prediction_events': 3, 'formation': [1, 5, 2, 3], 'transfer_limit': 4}
314 1.2
iteration  919
{'prediction_events': 5, 'from_scratch_prediction_events': 3, 'formation': [1, 5, 2, 3], 'transfer_limit': 11}
314 1.2
iteration  920
{'prediction_events': 5, 'from_scratch_prediction_events': 4, 'formation': [1, 5, 4, 1], 'transfer_limit': 1}
365 0.6
iteration  921
{'prediction_events': 5, 'from_scratch_prediction_events': 4, 'formation': [1, 5, 4, 1], 'transfer_limit': 2}
368 0.8
iteration  922
{'prediction_events': 5, 'from_scratch_prediction_events': 4, 'formation': [1, 5, 4, 1], 'transfer_limit': 3}
368 0.8
iteration  923
{'prediction_events': 5, 'from_scratch_prediction_events': 4, 'formation': [1, 5, 4, 1], 'transfer_limit': 4}
368 0.8
iteration  924
{'prediction_events': 5, 'from_scratch_prediction_events': 4, 'formation': [1, 5, 4, 1], 'transfer_limit': 11}
368 0.8
iteration  925
{'prediction_events': 5, 'from_scratch_predi

337 1.0
iteration  980
{'prediction_events': 5, 'from_scratch_prediction_events': 5, 'formation': [1, 3, 5, 2], 'transfer_limit': 1}
375 0.6
iteration  981
{'prediction_events': 5, 'from_scratch_prediction_events': 5, 'formation': [1, 3, 5, 2], 'transfer_limit': 2}
380 1.0
iteration  982
{'prediction_events': 5, 'from_scratch_prediction_events': 5, 'formation': [1, 3, 5, 2], 'transfer_limit': 3}
380 1.0
iteration  983
{'prediction_events': 5, 'from_scratch_prediction_events': 5, 'formation': [1, 3, 5, 2], 'transfer_limit': 4}
380 1.0
iteration  984
{'prediction_events': 5, 'from_scratch_prediction_events': 5, 'formation': [1, 3, 5, 2], 'transfer_limit': 11}
380 1.0
iteration  985
{'prediction_events': 5, 'from_scratch_prediction_events': 5, 'formation': [1, 4, 5, 1], 'transfer_limit': 1}
358 0.6
iteration  986
{'prediction_events': 5, 'from_scratch_prediction_events': 5, 'formation': [1, 4, 5, 1], 'transfer_limit': 2}
352 0.8
iteration  987
{'prediction_events': 5, 'from_scratch_predic

In [62]:
results_df = pd.DataFrame(results_arr)
results_df['formation'] = results_df['formation'].apply(str)

In [79]:
results_df[
    results_df['transfer_limit'] == 1
].groupby('formation')['total_points'].describe()

,count,mean,std,min,25%,50%,75%,max
formation,,,,,,,,
"[1, 3, 4, 3]",25.0,356.92,14.960838,310.0,354.0,365.0,365.0,369.0
"[1, 3, 5, 2]",25.0,355.96,15.759865,325.0,344.0,353.0,375.0,375.0
"[1, 4, 3, 3]",25.0,338.16,14.786480,301.0,329.0,337.0,346.0,362.0
"[1, 4, 4, 2]",25.0,346.44,13.668943,308.0,343.0,349.0,356.0,365.0
"[1, 4, 5, 1]",25.0,345.28,14.220056,321.0,338.0,353.0,358.0,368.0
"[1, 5, 2, 3]",25.0,339.32,14.194248,298.0,334.0,343.0,349.0,356.0
"[1, 5, 3, 2]",25.0,341.68,13.499136,319.0,329.0,345.0,350.0,368.0
"[1, 5, 4, 1]",25.0,353.32,13.936523,315.0,346.0,356.0,366.0,367.0


In [67]:
good_formations = [
    '[1, 3, 4, 3]',
    '[1, 3, 5, 2]',
    '[1, 5, 4, 1]'
]

In [69]:
results_df[
    results_df['formation'].isin(good_formations)
].groupby('transfer_limit')['total_points'].describe()

,count,mean,std,min,25%,50%,75%,max
transfer_limit,,,,,,,,
1,75.0,355.400000,14.781288,310.0,345.5,359.0,366.0,375.0
2,75.0,353.053333,15.611753,310.0,346.0,355.0,365.0,380.0
3,75.0,352.800000,15.782303,310.0,344.5,355.0,365.0,380.0
4,75.0,352.800000,15.782303,310.0,344.5,355.0,365.0,380.0
11,75.0,352.800000,15.782303,310.0,344.5,355.0,365.0,380.0


In [80]:
results_df[
    (results_df['formation'].isin(good_formations))
    & (results_df['transfer_limit'] == 1)
].groupby('from_scratch_prediction_events')['total_points'].describe()

,count,mean,std,min,25%,50%,75%,max
from_scratch_prediction_events,,,,,,,,
1,15.0,357.266667,11.234301,343.0,344.0,366.0,366.0,367.0
2,15.0,359.800000,6.247285,351.0,354.0,359.0,365.0,369.0
3,15.0,333.066667,11.125689,310.0,328.5,332.0,340.5,348.0
4,15.0,364.466667,7.836423,353.0,357.5,365.0,370.0,375.0
5,15.0,362.400000,10.245557,342.0,353.5,365.0,370.0,375.0


In [81]:
results_df[
    (results_df['formation'].isin(good_formations))
    & (results_df['transfer_limit'] == 1)
].groupby('prediction_events')['total_points'].describe()

,count,mean,std,min,25%,50%,75%,max
prediction_events,,,,,,,,
1,15.0,347.800000,17.280046,310.0,344.5,354.0,358.0,366.0
2,15.0,354.600000,16.321765,325.0,342.5,359.0,365.5,375.0
3,15.0,357.866667,13.468835,332.0,349.5,365.0,366.5,375.0
4,15.0,357.400000,13.436624,331.0,349.0,363.0,366.0,375.0
5,15.0,359.333333,11.745313,336.0,349.5,365.0,366.0,375.0


In [78]:
results_df[results_df['total_points'] >= 370].sort_values('total_points', ascending=False)

,formation,from_scratch_prediction_events,mean_transfers,prediction_events,total_points,transfer_limit
984,"[1, 3, 5, 2]",5,1.0,5,380,11
983,"[1, 3, 5, 2]",5,1.0,5,380,4
982,"[1, 3, 5, 2]",5,1.0,5,380,3
981,"[1, 3, 5, 2]",5,1.0,5,380,2
944,"[1, 3, 5, 2]",4,1.0,5,380,11
943,"[1, 3, 5, 2]",4,1.0,5,380,4
942,"[1, 3, 5, 2]",4,1.0,5,380,3
941,"[1, 3, 5, 2]",4,1.0,5,380,2
580,"[1, 3, 5, 2]",5,0.6,3,375,1
980,"[1, 3, 5, 2]",5,0.6,5,375,1
